ML Models : KNN,K-means,SVM

Feature importance : Linear Regression,Permutation,decision tree

In [88]:
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.inspection import permutation_importance
from sklearn.tree import DecisionTreeClassifier    
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
iris = datasets.load_iris()
x = iris.data
y = iris.target

In [89]:
def data_initial():
    df_original = pd.DataFrame(x)
    df_2feature = df_original.iloc[: , :-2]
    df_2feature[2] = y
    df_2feature.columns = ['x1','x2','y']
    df_2feature.loc[df_2feature.y==2,'y']=1

    subset = df_2feature.sample(frac=0.2,replace=False,random_state=43)
    df_2feature = df_2feature.append(subset)
    df_2feature = df_2feature.drop_duplicates(subset=['x1','x2','y'],keep=False)

    subsample_x = subset.iloc[:,:-1].values
    subsample_y = subset.iloc[:,-1].values

    error_a = []
    error_b = []
    for i in range(21) :
        error_a.append(i)
        error_b.append(20-i)

    return df_original,df_2feature,subset,subsample_x,subsample_y,error_a,error_b
    

In [90]:
df_original,df_2feature,subset,subsample_x,subsample_y,error_a,error_b = data_initial()
print(df_2feature.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71 entries, 3 to 148
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x1      71 non-null     float64
 1   x2      71 non-null     float64
 2   y       71 non-null     int64  
dtypes: float64(2), int64(1)
memory usage: 2.2 KB
None


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [91]:
def featureImportance_linearRegression(subx,suby):
    clf_subsample = LinearRegression()
    clf_subsample.fit(subx, suby)
    importance = clf_subsample.coef_

    acc = 0
    a = []
    b = []
    for i,v in enumerate(importance):
        b.append(v)
        #print(i,v)
        acc += abs(v)

    for i,v in enumerate(importance):
        a.append(abs(v)/acc)
        #print(abs(v)/acc)
    
    return a,b

In [92]:
def featureImportance_permutation(model,X,y) :
    # perform permutation importance 
    results = permutation_importance(model, X, y, scoring='accuracy')  
    # get importance 
    importance = results.importances_mean  
    # summarize feature importance 

    acc = 0
    a = []
    b = []
    for i,v in enumerate(importance):
        b.append(v)
        #print(i,v)
        acc += abs(v)

    for i,v in enumerate(importance):
        a.append(abs(v)/acc)
        #print(abs(v)/acc)
    
    return a,b
    

In [93]:
def featureImportance_DTree(subx,suby) :
    clf_subsample = DecisionTreeClassifier()
    clf_subsample.fit(subx, suby)
    importance = clf_subsample.feature_importances_

    acc = 0
    a = []
    b = []
    for i,v in enumerate(importance):
        b.append(v)
        #print(i,v)
        acc += abs(v)

    for i,v in enumerate(importance):
        a.append(abs(v)/acc)
        #print(abs(v)/acc)
    
    return a,b
    

In [94]:
importanceReg,RegCoef = featureImportance_linearRegression(subsample_x,subsample_y)
model = KNeighborsClassifier(n_neighbors=5,algorithm="brute")  
model.fit(subsample_x,subsample_y)  
importancePermu,PermuCoef = featureImportance_permutation(model,subsample_x,subsample_y)
importanceDTree,DTreeCoef = featureImportance_DTree(subsample_x,subsample_y)
print("RegCoef : " +str(RegCoef))
print("importanceReg : "+str(importanceReg))
print("PermuCoef : "+str(PermuCoef))
print("importancePermu : "+str(importancePermu))
print("DTreeCoef : " +str(DTreeCoef))
print("importanceDTree : "+str(importanceDTree))

RegCoef : [0.3166287926250703, -0.6168170867712136]
importanceReg : [0.33920423199023964, 0.6607957680097605]
PermuCoef : [0.42000000000000004, 0.07333333333333332]
importancePermu : [0.8513513513513514, 0.14864864864864863]
DTreeCoef : [0.7469339284617432, 0.2530660715382567]
importanceDTree : [0.7469339284617432, 0.2530660715382567]


In [95]:
def df_2feature_be_generate(BE,i,n,importance=importanceReg) :#i for brute force or not,n for negative importance
    temp = df_2feature.copy()
    df_2feature_be = pd.DataFrame.to_numpy(df_2feature)
    
    if i >= 0 and n == 0 :
        e1 = temp['x1'].mean()*BE*(error_a[i]/(error_a[i]+error_b[i]))
        e2 = temp['x2'].mean()*BE*(error_b[i]/(error_a[i]+error_b[i]))
    elif i == -1 and n == 1:
        e1 = temp['x1'].mean()*BE*importance[0]
        e2 = temp['x2'].mean()*BE*importance[1]
    elif i == -1 and n == 0 :
        e1 = temp['x1'].mean()*BE*importance[1]
        e2 = temp['x2'].mean()*BE*importance[0]
    
    for idx,cell in temp['x1'].iteritems():
        temp['x1'][idx] += e1
    temp = temp.to_numpy()
    df_2feature_be = np.vstack((df_2feature_be,temp))
    #print(df_2feature_be.shape)
    temp = df_2feature.copy()

    for idx,cell in temp['x1'].iteritems():
        temp['x1'][idx] = temp['x1'][idx] - e1
    temp = temp.to_numpy()
    df_2feature_be = np.vstack((df_2feature_be,temp))
    #print(df_2feature_be.shape)
    temp = df_2feature.copy()

    for idx,cell in temp['x2'].iteritems():
        temp['x2'][idx] += e2
    temp = temp.to_numpy()
    df_2feature_be = np.vstack((df_2feature_be,temp))
    #print(df_2feature_be.shape)
    temp = df_2feature.copy()

    for idx,cell in temp['x2'].iteritems():
        temp['x2'][idx] = temp['x2'][idx] - e2
    temp = temp.to_numpy()
    df_2feature_be = np.vstack((df_2feature_be,temp))
    #print(df_2feature_be.shape)
    temp = df_2feature.copy()

    df_2feature_be = pd.DataFrame(df_2feature_be)
    #print(df_2feature_be)
    return df_2feature_be

In [96]:
#brute force from 0-20 using svm
BE = 0.2
for i in range(21) :
    df_2feature_be = df_2feature_be_generate(BE,i,0)
    x_be = df_2feature_be.iloc[:, :-1].values
    y_be = df_2feature_be.iloc[:,-1].values
    X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
    clf_BE = SVC(kernel='linear',C=1,gamma='auto')
    clf_BE.fit(X_train, y_train)
    y_BE_predict = clf_BE.predict(X_test)
    result_BE_train = clf_BE.score(X_train, y_train)
    result_BE_test = clf_BE.score(X_test, y_test)
    subset_raw = clf_BE.score(subsample_x, subsample_y)
    print('BESVM Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str((error_a[i]/(error_a[i]+error_b[i])))+'),x2 with error ('+str(BE)+'%*'+str((error_b[i]/(error_a[i]+error_b[i])))+') : '+str(result_BE_train))
    print('BESVM Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str((error_a[i]/(error_a[i]+error_b[i])))+'),x2 with error ('+str(BE)+'%*'+str((error_b[i]/(error_a[i]+error_b[i])))+') : '+str(result_BE_test))
    print('BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))
    

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

BESVM Accuracy (on training) = with x1 bounded error (0.2%*0.0),x2 with error (0.2%*1.0) : 0.9577464788732394
BESVM Accuracy (on testing) = with x1 bounded error (0.2%*0.0),x2 with error (0.2%*1.0) : 0.9436619718309859
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 1.0
BESVM Accuracy (on training) = with x1 bounded error (0.2%*0.05),x2 with error (0.2%*0.95) : 0.9647887323943662
BESVM Accuracy (on testing) = with x1 bounded error (0.2%*0.05),x2 with error (0.2%*0.95) : 0.9436619718309859
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 1.0


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-

BESVM Accuracy (on training) = with x1 bounded error (0.2%*0.1),x2 with error (0.2%*0.9) : 0.9647887323943662
BESVM Accuracy (on testing) = with x1 bounded error (0.2%*0.1),x2 with error (0.2%*0.9) : 0.9436619718309859
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 1.0
BESVM Accuracy (on training) = with x1 bounded error (0.2%*0.15),x2 with error (0.2%*0.85) : 0.954225352112676
BESVM Accuracy (on testing) = with x1 bounded error (0.2%*0.15),x2 with error (0.2%*0.85) : 0.9436619718309859
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 1.0
BESVM Accuracy (on training) = with x1 bounded error (0.2%*0.2),x2 with error (0.2%*0.8) : 0.9471830985915493
BESVM Accuracy (on testing) = with x1 bounded error (0.2%*0.2),x2 with error (0.2%*0.8) : 0.9436619718309859
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 1.0


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

...

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

BESVM Accuracy (on training) = with x1 bounded error (0.2%*0.7),x2 with error (0.2%*0.3) : 0.9190140845070423
BESVM Accuracy (on testing) = with x1 bounded error (0.2%*0.7),x2 with error (0.2%*0.3) : 0.9436619718309859
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 1.0
BESVM Accuracy (on training) = with x1 bounded error (0.2%*0.75),x2 with error (0.2%*0.25) : 0.9154929577464789
BESVM Accuracy (on testing) = with x1 bounded error (0.2%*0.75),x2 with error (0.2%*0.25) : 0.9436619718309859
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 1.0
BESVM Accuracy (on training) = with x1 bounded error (0.2%*0.8),x2 with error (0.2%*0.2) : 0.9154929577464789
BESVM Accuracy (on testing) = with x1 bounded error (0.2%*0.8),x2 with error (0.2%*0.2) : 0.9154929577464789
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 1.0


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

BESVM Accuracy (on training) = with x1 bounded error (0.2%*0.85),x2 with error (0.2%*0.15) : 0.9154929577464789
BESVM Accuracy (on testing) = with x1 bounded error (0.2%*0.85),x2 with error (0.2%*0.15) : 0.9436619718309859
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9666666666666667
BESVM Accuracy (on training) = with x1 bounded error (0.2%*0.9),x2 with error (0.2%*0.1) : 0.9119718309859155
BESVM Accuracy (on testing) = with x1 bounded error (0.2%*0.9),x2 with error (0.2%*0.1) : 0.9295774647887324
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9666666666666667
BESVM Accuracy (on training) = with x1 bounded error (0.2%*0.95),x2 with error (0.2%*0.05) : 0.9119718309859155
BESVM Accuracy (on testing) = with x1 bounded error (0.2%*0.95),x2 with error (0.2%*0.05) : 0.9295774647887324
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9666666666666667
BESVM Accuracy (on training) = with x1 bounded error (0.2%*1.0

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

In [97]:
#using importanceReg as error allocation rate to predict by svm
df_2feature_be = df_2feature_be_generate(BE,-1,1,importanceReg)
x_be = df_2feature_be.iloc[:, :-1].values
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
clf_BE = SVC(kernel='linear',C=1,gamma='auto')
clf_BE.fit(X_train, y_train)
y_BE_predict = clf_BE.predict(X_test)
result_BE_train = clf_BE.score(X_train, y_train)
result_BE_test = clf_BE.score(X_test, y_test)
subset_raw = clf_BE.score(subsample_x, subsample_y)
print('BESVM Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str(importanceReg[0])+'),x2 with error ('+str(BE)+'%*'+str(importanceReg[1])+') : '+str(result_BE_train))
print('BESVM Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str(importanceReg[0])+'),x2 with error ('+str(BE)+'%*'+str(importanceReg[1])+') : '+str(result_BE_test))
print('BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))


BESVM Accuracy (on training) = with x1 bounded error (0.2%*0.33920423199023964),x2 with error (0.2%*0.6607957680097605) : 0.9330985915492958
BESVM Accuracy (on testing) = with x1 bounded error (0.2%*0.33920423199023964),x2 with error (0.2%*0.6607957680097605) : 0.9436619718309859
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 1.0


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

In [98]:
#using importancePermu as error allocation rate to predict by svm
df_2feature_be = df_2feature_be_generate(BE,-1,0,importancePermu)
x_be = df_2feature_be.iloc[:, :-1].values
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
clf_BE = SVC(kernel='linear',C=1,gamma='auto')
clf_BE.fit(X_train, y_train)
y_BE_predict = clf_BE.predict(X_test)
result_BE_train = clf_BE.score(X_train, y_train)
result_BE_test = clf_BE.score(X_test, y_test)
subset_raw = clf_BE.score(subsample_x, subsample_y)
print('BESVM Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str(importanceReg[0])+'),x2 with error ('+str(BE)+'%*'+str(importanceReg[1])+') : '+str(result_BE_train))
print('BESVM Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str(importanceReg[0])+'),x2 with error ('+str(BE)+'%*'+str(importanceReg[1])+') : '+str(result_BE_test))
print('BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

BESVM Accuracy (on training) = with x1 bounded error (0.2%*0.33920423199023964),x2 with error (0.2%*0.6607957680097605) : 0.954225352112676
BESVM Accuracy (on testing) = with x1 bounded error (0.2%*0.33920423199023964),x2 with error (0.2%*0.6607957680097605) : 0.9436619718309859
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 1.0


above are experiment using svm model,

below are experiment using KNN model

In [99]:
#using knn with brute force from 0-20
for i in range(21):
    df_2feature_be = df_2feature_be_generate(BE,i,0)
    x_be = df_2feature_be.iloc[:, :-1].values
    y_be = df_2feature_be.iloc[:,-1].values
    X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
    KNN_BE = KNeighborsClassifier()
    KNN_BE.fit(X_train,y_train)
    y_BE_predict = KNN_BE.predict(X_test)
    result_BE_train = KNN_BE.score(X_train, y_train)
    result_BE_test = KNN_BE.score(X_test, y_test)
    subset_raw = KNN_BE.score(subsample_x, subsample_y)
    
    print('BEKNN Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str((error_a[i]/(error_a[i]+error_b[i])))+'),x2 with error ('+str(BE)+'%*'+str((error_b[i]/(error_a[i]+error_b[i])))+') : '+str(result_BE_train))
    print('BEKNN Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str((error_a[i]/(error_a[i]+error_b[i])))+'),x2 with error ('+str(BE)+'%*'+str((error_b[i]/(error_a[i]+error_b[i])))+') : '+str(result_BE_test))
    print('BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))
    

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

BEKNN Accuracy (on training) = with x1 bounded error (0.2%*0.0),x2 with error (0.2%*1.0) : 0.9507042253521126
BEKNN Accuracy (on testing) = with x1 bounded error (0.2%*0.0),x2 with error (0.2%*1.0) : 0.9295774647887324
BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error 1.0


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-

BEKNN Accuracy (on training) = with x1 bounded error (0.2%*0.05),x2 with error (0.2%*0.95) : 0.9577464788732394
BEKNN Accuracy (on testing) = with x1 bounded error (0.2%*0.05),x2 with error (0.2%*0.95) : 0.9577464788732394
BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error 1.0
BEKNN Accuracy (on training) = with x1 bounded error (0.2%*0.1),x2 with error (0.2%*0.9) : 0.9471830985915493
BEKNN Accuracy (on testing) = with x1 bounded error (0.2%*0.1),x2 with error (0.2%*0.9) : 0.9577464788732394
BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error 1.0


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

...

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

BEKNN Accuracy (on training) = with x1 bounded error (0.2%*0.9),x2 with error (0.2%*0.1) : 0.926056338028169
BEKNN Accuracy (on testing) = with x1 bounded error (0.2%*0.9),x2 with error (0.2%*0.1) : 0.9295774647887324
BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error 1.0


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

BEKNN Accuracy (on training) = with x1 bounded error (0.2%*0.95),x2 with error (0.2%*0.05) : 0.9366197183098591
BEKNN Accuracy (on testing) = with x1 bounded error (0.2%*0.95),x2 with error (0.2%*0.05) : 0.9014084507042254
BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error 1.0
BEKNN Accuracy (on training) = with x1 bounded error (0.2%*1.0),x2 with error (0.2%*0.0) : 0.9401408450704225
BEKNN Accuracy (on testing) = with x1 bounded error (0.2%*1.0),x2 with error (0.2%*0.0) : 0.9154929577464789
BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error 1.0


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

In [100]:
#using importanceReg as error allocation rate to predict by knn
df_2feature_be = df_2feature_be_generate(BE,-1,0,importanceReg)
x_be = df_2feature_be.iloc[:, :-1].values
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
KNN_BE = KNeighborsClassifier()
KNN_BE.fit(X_train, y_train)
y_BE_predict = KNN_BE.predict(X_test)
result_BE_train = KNN_BE.score(X_train, y_train)
result_BE_test = KNN_BE.score(X_test, y_test)
subset_raw = KNN_BE.score(subsample_x, subsample_y)
print('BEKNN Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str(importanceReg[0])+'),x2 with error ('+str(BE)+'%*'+str(importanceReg[1])+') : '+str(result_BE_train))
print('BEKNN Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str(importanceReg[0])+'),x2 with error ('+str(BE)+'%*'+str(importanceReg[1])+') : '+str(result_BE_test))
print('BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


BEKNN Accuracy (on training) = with x1 bounded error (0.2%*0.33920423199023964),x2 with error (0.2%*0.6607957680097605) : 0.9295774647887324
BEKNN Accuracy (on testing) = with x1 bounded error (0.2%*0.33920423199023964),x2 with error (0.2%*0.6607957680097605) : 0.9436619718309859
BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [101]:
#using importancePermu as error allocation rate to predict by knn
df_2feature_be = df_2feature_be_generate(BE,-1,0,importancePermu)
x_be = df_2feature_be.iloc[:, :-1].values
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
KNN_BE = KNeighborsClassifier()
KNN_BE.fit(X_train, y_train)
y_BE_predict = KNN_BE.predict(X_test)
result_BE_train = KNN_BE.score(X_train, y_train)
result_BE_test = KNN_BE.score(X_test, y_test)
subset_raw = KNN_BE.score(subsample_x, subsample_y)
print('BEKNN Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str(importancePermu[1])+'),x2 with error ('+str(BE)+'%*'+str(importancePermu[0])+') : '+str(result_BE_train))
print('BEKNN Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str(importancePermu[1])+'),x2 with error ('+str(BE)+'%*'+str(importancePermu[0])+') : '+str(result_BE_test))
print('BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

BEKNN Accuracy (on training) = with x1 bounded error (0.2%*0.14864864864864863),x2 with error (0.2%*0.8513513513513514) : 0.9507042253521126
BEKNN Accuracy (on testing) = with x1 bounded error (0.2%*0.14864864864864863),x2 with error (0.2%*0.8513513513513514) : 0.9436619718309859
BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error 1.0


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [102]:
#using importanceDTree as error allocation rate to predict by svm
df_2feature_be = df_2feature_be_generate(BE,-1,0,importanceDTree)
x_be = df_2feature_be.iloc[:, :-1].values
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
clf_BE = SVC(kernel='linear',C=1,gamma='auto')
clf_BE.fit(X_train, y_train)
y_BE_predict = clf_BE.predict(X_test)
result_BE_train = clf_BE.score(X_train, y_train)
result_BE_test = clf_BE.score(X_test, y_test)
subset_raw = clf_BE.score(subsample_x, subsample_y)
print('BESVM Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str(importanceDTree[1])+'),x2 with error ('+str(BE)+'%*'+str(importanceDTree[0])+') : '+str(result_BE_train))
print('BESVM Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str(importanceDTree[1])+'),x2 with error ('+str(BE)+'%*'+str(importanceDTree[0])+') : '+str(result_BE_test))
print('BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

BESVM Accuracy (on training) = with x1 bounded error (0.2%*0.2530660715382567),x2 with error (0.2%*0.7469339284617432) : 0.9436619718309859
BESVM Accuracy (on testing) = with x1 bounded error (0.2%*0.2530660715382567),x2 with error (0.2%*0.7469339284617432) : 0.9436619718309859
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 1.0


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [103]:
#using importanceDTree as error allocation rate to predict by knn
df_2feature_be = df_2feature_be_generate(BE,-1,0,importanceDTree)
x_be = df_2feature_be.iloc[:, :-1].values
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
KNN_BE = KNeighborsClassifier()
KNN_BE.fit(X_train, y_train)
y_BE_predict = KNN_BE.predict(X_test)
result_BE_train = KNN_BE.score(X_train, y_train)
result_BE_test = KNN_BE.score(X_test, y_test)
subset_raw = KNN_BE.score(subsample_x, subsample_y)
print('BEKNN Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str(importanceDTree[1])+'),x2 with error ('+str(BE)+'%*'+str(importanceDTree[0])+') : '+str(result_BE_train))
print('BEKNN Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str(importanceDTree[1])+'),x2 with error ('+str(BE)+'%*'+str(importanceDTree[0])+') : '+str(result_BE_test))
print('BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

BEKNN Accuracy (on training) = with x1 bounded error (0.2%*0.2530660715382567),x2 with error (0.2%*0.7469339284617432) : 0.9401408450704225
BEKNN Accuracy (on testing) = with x1 bounded error (0.2%*0.2530660715382567),x2 with error (0.2%*0.7469339284617432) : 0.9295774647887324
BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error 1.0


three ways of importance measurement -> K-Means
trust confidence as importance measurement

In [104]:
#using kMeans with brute force from 0-20
for i in range(21):
    df_2feature_be = df_2feature_be_generate(BE,i,0)
    x_be = df_2feature_be.iloc[:, :-1].values
    y_be = df_2feature_be.iloc[:,-1].values
    X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
    KM_BE=KMeans(n_clusters=2,init='random',random_state=5)
    KM_BE.fit(X_train)
    y_BE_predict = KM_BE.predict(X_train)
    y_BE_predict_test = KM_BE.predict(X_test)
    y_subset = KM_BE.predict(subsample_x)
    result_BE_train = 1 - np.sum(y_BE_predict!=y_train)/len(y_train)
    #print(score)
    result_BE_test = 1 - np.sum(y_BE_predict_test!=y_train)/len(y_test)
    subset_raw = 1 - np.sum(subsample_y!=y_subset)/len(subsample_y)
    #print(score)
    #result_BE_train = KM_BE.score(X_train, y_train)
    #result_BE_test = KM_BE.score(X_test, y_test)
    #subset_raw = KM_BE.score(subsample_x, subsample_y)
    
    
    print('BEKM Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str((error_a[i]/(error_a[i]+error_b[i])))+'),x2 with error ('+str(BE)+'%*'+str((error_b[i]/(error_a[i]+error_b[i])))+') : '+str(result_BE_train))
    print('BEKM Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str((error_a[i]/(error_a[i]+error_b[i])))+'),x2 with error ('+str(BE)+'%*'+str((error_b[i]/(error_a[i]+error_b[i])))+') : '+str(result_BE_test))
    print('BEKM Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))
    

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

BEKM Accuracy (on training) = with x1 bounded error (0.2%*0.0),x2 with error (0.2%*1.0) : 0.7464788732394366
BEKM Accuracy (on testing) = with x1 bounded error (0.2%*0.0),x2 with error (0.2%*1.0) : 0.9859154929577465
BEKM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.8
BEKM Accuracy (on training) = with x1 bounded error (0.2%*0.05),x2 with error (0.2%*0.95) : 0.7394366197183099
BEKM Accuracy (on testing) = with x1 bounded error (0.2%*0.05),x2 with error (0.2%*0.95) : 0.9859154929577465
BEKM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.8


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:30: 

BEKM Accuracy (on training) = with x1 bounded error (0.2%*0.1),x2 with error (0.2%*0.9) : 0.7429577464788732
BEKM Accuracy (on testing) = with x1 bounded error (0.2%*0.1),x2 with error (0.2%*0.9) : 0.9859154929577465
BEKM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.8
BEKM Accuracy (on training) = with x1 bounded error (0.2%*0.15),x2 with error (0.2%*0.85) : 0.7429577464788732
BEKM Accuracy (on testing) = with x1 bounded error (0.2%*0.15),x2 with error (0.2%*0.85) : 0.9859154929577465
BEKM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.8


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:30: 

...

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:30: 

BEKM Accuracy (on training) = with x1 bounded error (0.2%*0.8),x2 with error (0.2%*0.2) : 0.6866197183098591
BEKM Accuracy (on testing) = with x1 bounded error (0.2%*0.8),x2 with error (0.2%*0.2) : 0.9859154929577465
BEKM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.8
BEKM Accuracy (on training) = with x1 bounded error (0.2%*0.85),x2 with error (0.2%*0.15) : 0.6830985915492958
BEKM Accuracy (on testing) = with x1 bounded error (0.2%*0.85),x2 with error (0.2%*0.15) : 0.9859154929577465
BEKM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.8


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:30: 

BEKM Accuracy (on training) = with x1 bounded error (0.2%*0.9),x2 with error (0.2%*0.1) : 0.6866197183098591
BEKM Accuracy (on testing) = with x1 bounded error (0.2%*0.9),x2 with error (0.2%*0.1) : 0.9859154929577465
BEKM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.8
BEKM Accuracy (on training) = with x1 bounded error (0.2%*0.95),x2 with error (0.2%*0.05) : 0.6830985915492958
BEKM Accuracy (on testing) = with x1 bounded error (0.2%*0.95),x2 with error (0.2%*0.05) : 0.9859154929577465
BEKM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.8
BEKM Accuracy (on training) = with x1 bounded error (0.2%*1.0),x2 with error (0.2%*0.0) : 0.6795774647887324
BEKM Accuracy (on testing) = with x1 bounded error (0.2%*1.0),x2 with error (0.2%*0.0) : 0.9859154929577465
BEKM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.8


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


In [105]:
#importanceReg,KMeans
df_2feature_be = df_2feature_be_generate(BE,-1,1,importanceReg)
x_be = df_2feature_be.iloc[:, :-1].values
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
KM_BE=KMeans(n_clusters=2,init='random',random_state=5)
KM_BE.fit(X_train)
y_BE_predict = KM_BE.predict(X_train)
y_BE_predict_test = KM_BE.predict(X_test)
y_subset = KM_BE.predict(subsample_x)
result_BE_train = 1 - np.sum(y_BE_predict!=y_train)/len(y_train)
result_BE_test = 1 - np.sum(y_BE_predict_test!=y_train)/len(y_test)
subset_raw = 1 - np.sum(subsample_y!=y_subset)/len(subsample_y)
    
    
print('BEKNN Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str(importanceReg[0])+'),x2 with error ('+str(BE)+'%*'+str(importanceReg[1])+') : '+str(result_BE_train))
print('BEKNN Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str(importanceReg[0])+'),x2 with error ('+str(BE)+'%*'+str(importanceReg[1])+') : '+str(result_BE_test))
print('BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

BEKNN Accuracy (on training) = with x1 bounded error (0.2%*0.33920423199023964),x2 with error (0.2%*0.6607957680097605) : 0.7288732394366197
BEKNN Accuracy (on testing) = with x1 bounded error (0.2%*0.33920423199023964),x2 with error (0.2%*0.6607957680097605) : 0.9859154929577465
BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.8


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if sys.path[0] == '':


In [106]:
#importancePermu,KMeans
df_2feature_be = df_2feature_be_generate(BE,-1,0,importancePermu)
x_be = df_2feature_be.iloc[:, :-1].values
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
KM_BE=KMeans(n_clusters=2,init='random',random_state=5)
KM_BE.fit(X_train)
y_BE_predict = KM_BE.predict(X_train)
y_BE_predict_test = KM_BE.predict(X_test)
y_subset = KM_BE.predict(subsample_x)
result_BE_train = 1 - np.sum(y_BE_predict!=y_train)/len(y_train)
result_BE_test = 1 - np.sum(y_BE_predict_test!=y_train)/len(y_test)
subset_raw = 1 - np.sum(subsample_y!=y_subset)/len(subsample_y)
    
    
print('BEKNN Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str(importancePermu[1])+'),x2 with error ('+str(BE)+'%*'+str(importancePermu[0])+') : '+str(result_BE_train))
print('BEKNN Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str(importancePermu[1])+'),x2 with error ('+str(BE)+'%*'+str(importancePermu[0])+') : '+str(result_BE_test))
print('BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

BEKNN Accuracy (on training) = with x1 bounded error (0.2%*0.14864864864864863),x2 with error (0.2%*0.8513513513513514) : 0.7429577464788732
BEKNN Accuracy (on testing) = with x1 bounded error (0.2%*0.14864864864864863),x2 with error (0.2%*0.8513513513513514) : 0.9859154929577465
BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.8


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if sys.path[0] == '':


In [107]:
#importanceDTree,KMeans
df_2feature_be = df_2feature_be_generate(BE,-1,0,importanceDTree)
x_be = df_2feature_be.iloc[:, :-1].values
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
KM_BE=KMeans(n_clusters=2,init='random',random_state=5)
KM_BE.fit(X_train)
y_BE_predict = KM_BE.predict(X_train)
y_BE_predict_test = KM_BE.predict(X_test)
y_subset = KM_BE.predict(subsample_x)
result_BE_train = 1 - np.sum(y_BE_predict!=y_train)/len(y_train)
result_BE_test = 1 - np.sum(y_BE_predict_test!=y_train)/len(y_test)
subset_raw = 1 - np.sum(subsample_y!=y_subset)/len(subsample_y)
    
    
print('BEKNN Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str(importanceDTree[1])+'),x2 with error ('+str(BE)+'%*'+str(importanceDTree[0])+') : '+str(result_BE_train))
print('BEKNN Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str(importanceDTree[1])+'),x2 with error ('+str(BE)+'%*'+str(importanceDTree[0])+') : '+str(result_BE_test))
print('BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

BEKNN Accuracy (on training) = with x1 bounded error (0.2%*0.2530660715382567),x2 with error (0.2%*0.7469339284617432) : 0.7323943661971831
BEKNN Accuracy (on testing) = with x1 bounded error (0.2%*0.2530660715382567),x2 with error (0.2%*0.7469339284617432) : 0.9859154929577465
BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.8


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if sys.path[0] == '':


In [109]:
model = KMeans(n_clusters=2,init='random',random_state=5)
model.fit(subsample_x)  
importancePermu,PermuCoef = featureImportance_permutation(model,subsample_x,subsample_y)

#importancePermu,KMeans
df_2feature_be = df_2feature_be_generate(BE,-1,0,importancePermu)
x_be = df_2feature_be.iloc[:, :-1].values
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
KM_BE=KMeans(n_clusters=2,init='random',random_state=5)
KM_BE.fit(X_train)
y_BE_predict = KM_BE.predict(X_train)
y_BE_predict_test = KM_BE.predict(X_test)
y_subset = KM_BE.predict(subsample_x)
result_BE_train = 1 - np.sum(y_BE_predict!=y_train)/len(y_train)
result_BE_test = 1 - np.sum(y_BE_predict_test!=y_train)/len(y_test)
subset_raw = 1 - np.sum(subsample_y!=y_subset)/len(subsample_y)
    
    
print('BEKNN Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str(importancePermu[1])+'),x2 with error ('+str(BE)+'%*'+str(importancePermu[0])+') : '+str(result_BE_train))
print('BEKNN Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str(importancePermu[1])+'),x2 with error ('+str(BE)+'%*'+str(importancePermu[0])+') : '+str(result_BE_test))
print('BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))

BEKNN Accuracy (on training) = with x1 bounded error (0.2%*0.16071428571428567),x2 with error (0.2%*0.8392857142857143) : 0.7429577464788732
BEKNN Accuracy (on testing) = with x1 bounded error (0.2%*0.16071428571428567),x2 with error (0.2%*0.8392857142857143) : 0.9859154929577465
BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.8


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm